# import

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('../data/w_review_train.csv',header=None)
train.columns = ['review']

# explore

In [3]:
train.sample(10,random_state=0)

,review
12836,ในวันที่เร่งรีบ ไม่รู้จะกินอะไรมองหาร้านที่คนน...
10913,รอรถไฟกลับนครปฐมค่ะ ฝนตกพรำๆตลอดที่มาที่นี่ พอ...
4214,คราวนี้ได้มีโอกาสมาลองร้านไอติมเปิดใหม่ที่ส่งม...
8198,ร้านนี้โดดเด่นด้วยป้ายบอกร้านที่สูงลิบลิ่ว และ...
31403,ร้านนี้อยู่ตรงหน้า 7-11 ที่เป็นแหล่งรวมรถเข็นข...
13917,เป็นร้านที่มีบรรยากาศค่อนข้างสวย ตบแต่งมีสไตล...
27440,สเต็กโชกุนอยู่ริมถนนเพชรเกษม เกือบถึงซอยวัดม่ว...
11667,ชอบลูกชิ้นปลาร้านนี้ สั่งเย็นตาโฟมากิน รสชาติเ...
29616,ร้านเย็นตาโฟในตำนานของ ม.นเรศวร\nรอบนี้มาลองสั...
39864,เวลามีเพื่อน หรือญาติผู้ใหญ่แวะมาเยี่ยมเยียน เ...


In [4]:
(train.shape)

(40000, 1)

**sample**

In [6]:
import numpy as np

np.random.seed(1)
rand_ind = np.random.randint(low = 0, high = train.shape[0], size = 5)

for ind in rand_ind:
    print(f"{ind} : {train.iloc[ind].values[0]} \n")

SyntaxError: invalid syntax (<ipython-input-6-8060b2774adc>, line 7)

In [ ]:
from pythainlp.tokenize import word_tokenize

In [ ]:
# REDUCE SIZE TRAIN
train = train.head(1000).copy()

In [ ]:
train['word_tokenized'] = train['review'].apply(lambda x: word_tokenize(x,engine='attacut',keep_whitespace=False))

In [ ]:
train.head()

In [ ]:
train['len_review'] = train['word_tokenized'].apply(lambda x: len(x))

In [ ]:
train['word_tokenized_uniq'] = train['word_tokenized'].apply(lambda x: list(set(x)))

In [ ]:
train['len_review_uniq'] = train['word_tokenized_uniq'].apply(lambda x: len(x))

In [ ]:
special_char = '''
~:""+[\\@^%(-"*|&<`._=]!>;?#$)/,ๆ'
'''

In [ ]:
train['word_tokenized_uniq_cls'] = train['word_tokenized_uniq'].apply(lambda x : [w for w in x if w not in special_char])

In [ ]:
train['len_review_uniq_cls'] = train['word_tokenized_uniq_cls'].apply(lambda x: len(x))

In [ ]:
thai_stopword_ls = list(pythainlp.corpus.thai_stopwords())

In [ ]:
train['word_tokenized_uniq_cls_revstopw'] = train['word_tokenized_uniq_cls'].apply(lambda x : [w for w in x if w not in thai_stopword_ls])

In [ ]:
train['len_review_uniq_cls_revstopw'] = train['word_tokenized_uniq_cls_revstopw'].apply(lambda x: len(x))

In [ ]:
train[['len_review','len_review_uniq','len_review_uniq_cls','len_review_uniq_cls_revstopw']].describe().transpose()

In [ ]:
print(train.loc[train['len_review_uniq_cls_revstopw'] == 16]['review'].values)
print('\n')
print(train.loc[train['len_review_uniq_cls_revstopw'] == 16]['word_tokenized_uniq_cls_revstopw'].iloc[0])

In [ ]:
print(train.loc[train['len_review_uniq_cls_revstopw'] == 303]['review'].values)
print('\n')
print(train.loc[train['len_review_uniq_cls_revstopw'] == 303]['word_tokenized_uniq_cls_revstopw'].iloc[0])

In [ ]:
# sampling
ind = 16
print(train.iloc[ind]['word_tokenized'])
print('\n')
print(train.iloc[ind]['word_tokenized_uniq'])
print('\n')
print([word for word in train.iloc[ind]['word_tokenized_uniq'] if word not in special_char and word not in thai_stopword_ls])

In [ ]:
# sampling
ind = 20
print(train.iloc[ind]['word_tokenized'])
print('\n')
print(train.iloc[ind]['word_tokenized_uniq'])
print('\n')
print([word for word in train.iloc[ind]['word_tokenized_uniq'] if word not in special_char])
print('\n')
print([correct(word) for word in train.iloc[ind]['word_tokenized_uniq'] if word not in special_char])
# special characther will be replace by "ใน"

# bag or word

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [ ]:
processed_docs = train['word_tokenized_uniq_cls_revstopw'].copy()

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
type(dictionary)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[500]

In [ ]:
bow_doc_4310 = bow_corpus[500]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2, random_state = 0)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('\n')
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf,
                                             num_topics=30, id2word=dictionary, passes=20, workers=4, random_state=0)

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('\n')
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
for ind in range(0,500,75):
    print("*"*30,ind,"*"*30)
    print([train['review'][ind]])
    for index, score in sorted(lda_model_tfidf[bow_corpus[ind]], key=lambda tup: -1*tup[1]):
        print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 20)))

In [ ]:
# FULL LOOP
# chunk_ind = list(range(0,train.shape[0],4000))
# chunk_ind

# import tqdm

# for ind in tqdm.tqdm(chunk_ind):
#     train_ = train.iloc[chunk_ind[ind]:chunk_ind[ind+1]].copy()
#     train_ = train_['review'].apply(lambda x: word_tokenize(x,engine='deepcut'))
#     train_.to_csv(f'../data/interim/train_tokenized_deepcut_chnk{ind}.csv', index=False)
#     del train_